## parse dataset

In [72]:
from datasets import load_dataset # Import dataset import function for hugging face
dataset = load_dataset("surrey-nlp/PLOD-CW") # import the coursework dataset from

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

import evaluate
seqeval = evaluate.load("seqeval")

import numpy as np


In [73]:
train_dict = dataset["train"]
train_tokens = train_dict["tokens"]
train_pos_tags = train_dict["pos_tags"]
train_ner_tags = train_dict["ner_tags"]

validation_dict = dataset["validation"]
validation_tokens = validation_dict["tokens"]
validation_pos_tags = validation_dict["pos_tags"]
validation_ner_tags = validation_dict["ner_tags"]

test_dict = dataset["test"]
test_tokens = test_dict["tokens"]
test_pos_tags = test_dict["pos_tags"]
test_ner_tags = test_dict["ner_tags"]

def data_to_lower(data:list[list[str]]) -> list[list[str]]:
    return [[token.lower() for token in tokens] for tokens in data]

train_tokens = data_to_lower(train_tokens)
validation_tokens = data_to_lower(validation_tokens)
test_tokens = data_to_lower(test_tokens)

class DataItem:
    def __init__(self, tokens, pos, ner, idx=0):
        self.idx=idx
        self.tokens:list[str] = tokens
        self.pos:list[str] = pos
        self.ner:list = ner
        self.tokenised_inputs:list = tokenizer(self.tokens, is_split_into_words=True)

    def get_as_tuple(self) -> tuple:
        return (self.tokens, self.pos, self.ner)
    
    def get_as_tuple_list(self) -> list[tuple]:
        tuple_list = []
        for idx in range(len(self.tokens)-1):
            tuple_list.append((self.tokens[idx], self.pos[idx], self.ner[idx]))
        return tuple_list
    
    def ner_label2idx(self, label2idx_dict):
        if not isinstance(self.ner[0], str):
            print("WARNING - NER not listed as labels! NER Type: ",type(self.ner[0]),", Exiting...")
            return
        for idx, ner in enumerate(self.ner):
            ner[idx] = label2idx_dict[ner]
    
    def ner_idx2label(self, idx2label_dict):
        if not isinstance(self.ner[0], int):
            print("WARNING - NER not listed as indecies! Exiting...")
            return
        for idx, ner in enumerate(self.ner):
            ner[idx] = idx2label_dict[ner]

class DataCollection:
    def __init__(self, data_collection:list[DataItem], max_token_length=512):
        self.max_token_length = max_token_length
        self.data_collection:list[DataItem] = data_collection
        self.unique_tags = self.get_unique_tags()
        self.item_embeddings:dict = self.create_item_embeddings(self.unique_tags)
        self.reverse_embeddings:dict = {v:k for k,v in self.item_embeddings.items()}

    def get_token_list(self) -> list[list[str]]:
        return [data_item.tokens for data_item in self.data_collection]

    def get_pos_list(self) -> list[list[str]]:
        return [data_item.pos for data_item in self.data_collection]

    def get_ner_list(self) -> list[list[str]]:
        return [data_item.ner for data_item in self.data_collection]
    
    def get_ner_idx_list(self) -> list[list[str]]:
        ner_idx_list_collection = []
        for data_item in self.data_collection:
            ner_idx_list = []
            for ner_tag in data_item.ner:
                ner_idx_list.append(self.item_embeddings[ner_tag])
            ner_idx_list_collection.append(ner_idx_list)
        return ner_idx_list_collection

    def get_unique_tags(self) -> list[str]:
        unique_list = []
        ner_tags_list:list = self.get_ner_list()
        for ner_list in ner_tags_list:
            for ner in ner_list:
                if ner not in unique_list:
                    unique_list.append(ner)
        return unique_list
    
    def create_item_embeddings(self, tags:list[str]) -> dict:
        return {label:idx for idx, label in enumerate(tags)}
    
    def get_invalid_token_lengths(self) -> list[int]:
        invalid_lengths = []
        for idx, data_item in enumerate(self.data_collection):
            if len(data_item.tokenised_inputs["input_ids"]) >= self.max_token_length:
                invalid_lengths.append(idx)
                print("Data item idx ",idx," has tokens longer than ",self.max_token_length)
        return invalid_lengths

    def remove_invalid_token_length_items(self) -> None:
        invalid_lengths = self.get_invalid_token_lengths()
        for index in sorted(invalid_lengths, reverse=True):
            del self.data_collection[index]

train_data:list[DataItem] = []
for idx in range(len(train_tokens)):
    train_data.append(DataItem(train_tokens[idx], train_pos_tags[idx], train_ner_tags[idx], idx))
train_collection:DataCollection = DataCollection(train_data)

validation_data:list[DataItem] = []
for idx in range(len(validation_tokens)):
    validation_data.append(DataItem(validation_tokens[idx], validation_pos_tags[idx], validation_ner_tags[idx], idx))
validation_collection:DataCollection = DataCollection(validation_data)

test_data:list[DataItem] = []
for idx in range(len(test_tokens)):
    test_data.append(DataItem(test_tokens[idx], test_pos_tags[idx], test_ner_tags[idx], idx))
test_collection:DataCollection = DataCollection(test_data)

In [74]:
train_collection.remove_invalid_token_length_items()
validation_collection.remove_invalid_token_length_items()
test_collection.remove_invalid_token_length_items()

for data_item in train_collection.data_collection:
    print("IDX:", data_item.idx)

Data item idx  286  has tokens longer than  512
IDX: 0
IDX: 1
IDX: 2
IDX: 3
IDX: 4
IDX: 5
IDX: 6
IDX: 7
IDX: 8
IDX: 9
IDX: 10
IDX: 11
IDX: 12
IDX: 13
IDX: 14
IDX: 15
IDX: 16
IDX: 17
IDX: 18
IDX: 19
IDX: 20
IDX: 21
IDX: 22
IDX: 23
IDX: 24
IDX: 25
IDX: 26
IDX: 27
IDX: 28
IDX: 29
IDX: 30
IDX: 31
IDX: 32
IDX: 33
IDX: 34
IDX: 35
IDX: 36
IDX: 37
IDX: 38
IDX: 39
IDX: 40
IDX: 41
IDX: 42
IDX: 43
IDX: 44
IDX: 45
IDX: 46
IDX: 47
IDX: 48
IDX: 49
IDX: 50
IDX: 51
IDX: 52
IDX: 53
IDX: 54
IDX: 55
IDX: 56
IDX: 57
IDX: 58
IDX: 59
IDX: 60
IDX: 61
IDX: 62
IDX: 63
IDX: 64
IDX: 65
IDX: 66
IDX: 67
IDX: 68
IDX: 69
IDX: 70
IDX: 71
IDX: 72
IDX: 73
IDX: 74
IDX: 75
IDX: 76
IDX: 77
IDX: 78
IDX: 79
IDX: 80
IDX: 81
IDX: 82
IDX: 83
IDX: 84
IDX: 85
IDX: 86
IDX: 87
IDX: 88
IDX: 89
IDX: 90
IDX: 91
IDX: 92
IDX: 93
IDX: 94
IDX: 95
IDX: 96
IDX: 97
IDX: 98
IDX: 99
IDX: 100
IDX: 101
IDX: 102
IDX: 103
IDX: 104
IDX: 105
IDX: 106
IDX: 107
IDX: 108
IDX: 109
IDX: 110
IDX: 111
IDX: 112
IDX: 113
IDX: 114
IDX: 115
IDX: 116
IDX: 117


In [75]:
# Following https://huggingface.co/docs/transformers/main/en/tasks/token_classification

## create label2id functionality

In [76]:
def print_lists_side_by_side(a_list, b_list):
    for i in range(max(len(b_list), len(a_list))):
        try:
            print(f"{a_list[i]}\t", end="")
        except IndexError:
            print(f" \t", end="")
        try:
            print(f"{b_list[i]}\t")
        except IndexError:
            print(f" \t")

def get_unique_labels(data:list[DataItem]) -> list[str]:
    return list(set([ner for item in data for ner in item.ner]))
    # return [
    #     ner # returned item to list
    #     for item in data # original list to item
    #     for ner in item.ner # inner list to out list
    # ]

unique_labels = get_unique_labels(train_data)

def create_label_index(labels:list) -> dict:
    return {label:idx for idx, label in enumerate(labels)}

label_index = create_label_index(unique_labels)
label_index

def labels2ids(labels:list[str], label_index:dict) -> list[int]:
    id_list:list[int] = []
    for label in labels:
        if label in label_index.keys():
            id_list.append(label_index[label])
        else:
            id_list.append(None)
    return id_list

def ids2labels(ids:list[int], label_index:dict) -> list[str]:
    label_list:list[int] = []
    for id in ids:
        label_list.append(list(label_index.keys())[list(label_index.values()).index(id)])
    return label_list

print(label_index)
ner_tags_test = train_data[0].ner
ner_to_ids_test = labels2ids(train_data[0].ner, label_index)
print_lists_side_by_side(ner_tags_test, ner_to_ids_test)
ids_to_ner_test = ids2labels(ner_to_ids_test, label_index)
ids_to_ner_test

{'B-LF': 0, 'B-AC': 1, 'B-O': 2, 'I-LF': 3}
B-O	2	
B-O	2	
B-O	2	
B-O	2	
B-LF	0	
I-LF	3	
I-LF	3	
I-LF	3	
I-LF	3	
B-O	2	
B-AC	1	
B-O	2	
B-O	2	
B-O	2	
B-O	2	


['B-O',
 'B-O',
 'B-O',
 'B-O',
 'B-LF',
 'I-LF',
 'I-LF',
 'I-LF',
 'I-LF',
 'B-O',
 'B-AC',
 'B-O',
 'B-O',
 'B-O',
 'B-O']

## Tokenization Example

Adds extra start and end tags (CLS and SEP), as well potentially splits one word into 2. Thus have to realign indecies.

We also have to assign -100 to CLS and SEP so they are ignored by PyTorch loss function (CrossEntropyLoss)

Only label first token of a word, add -100 for subtokens of the same word

In [77]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print(tokenized_input)

['[CLS]', 'for', 'this', 'purpose', 'the', 'gothenburg', 'young', 'persons', 'empowerment', 'scale', '(', 'g', '##ype', '##s', ')', 'was', 'developed', '.', '[SEP]']
{'input_ids': [101, 2005, 2023, 3800, 1996, 22836, 2402, 5381, 23011, 4094, 1006, 1043, 18863, 2015, 1007, 2001, 2764, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [78]:
def tokenize_and_align_labels(data_collection:DataCollection):
    tokenized_inputs = tokenizer(data_collection.get_token_list(), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(data_collection.get_ner_idx_list()):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = tokenize_and_align_labels(train_collection)
tokenized_validation = tokenize_and_align_labels(validation_collection)
tokenized_test = tokenize_and_align_labels(test_collection)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [79]:
label_list = train_collection.unique_tags
labels = train_collection.unique_tags


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

id2label = train_collection.reverse_embeddings
label2id = train_collection.item_embeddings

In [80]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
def turn_dict_to_list_of_dict(d):
    new_list = []

    for labels, inputs in zip(d["labels"], d["input_ids"]):
        entry = {"input_ids": inputs, "labels": labels}
        new_list.append(entry)

    return new_list

tokenised_train = turn_dict_to_list_of_dict(tokenized_train)
tokenised_val = turn_dict_to_list_of_dict(tokenized_validation)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test)

In [82]:
training_args = TrainingArguments(
    output_dir="distilbert_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=['none'], # REQUIRED because otherwise keeps asking to log into "wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/134 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.5216518044471741, 'eval_precision': 0.8701026608003353, 'eval_recall': 0.8307661532306462, 'eval_f1': 0.8499795333606223, 'eval_accuracy': 0.8422, 'eval_runtime': 0.2627, 'eval_samples_per_second': 582.502, 'eval_steps_per_second': 38.072, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6062090396881104, 'eval_precision': 0.8744166313109886, 'eval_recall': 0.8245649129825965, 'eval_f1': 0.8487593946257592, 'eval_accuracy': 0.8254, 'eval_runtime': 0.1861, 'eval_samples_per_second': 822.22, 'eval_steps_per_second': 53.74, 'epoch': 2.0}
{'train_runtime': 8.4355, 'train_samples_per_second': 253.925, 'train_steps_per_second': 15.885, 'train_loss': 0.3820226014550053, 'epoch': 2.0}


TrainOutput(global_step=134, training_loss=0.3820226014550053, metrics={'train_runtime': 8.4355, 'train_samples_per_second': 253.925, 'train_steps_per_second': 15.885, 'train_loss': 0.3820226014550053, 'epoch': 2.0})